In [1]:
%matplotlib inline
from dimod import BQM
from neal import SimulatedAnnealingSampler
import pandas as pd
import numpy as np
import rustworkx as rx
from rustworkx.visualization import mpl_draw
import matplotlib.pyplot as plt

# Setup Data

First of all, we need to setup all the distances for each point in the graph.

![problem graph](./assets/graph.png)

In [2]:
idx = pd.Index(['A', 'B', 'C', 'D', 'E'])
df = pd.DataFrame([
        np.array([0, 5.8, 5.8, 7.6, 8.8], dtype=float),
        np.array([5.8, 0, 2.4, 4.3, 4.3], dtype=float),
        np.array([5.8, 2.4, 0, 3.5, 4.7], dtype=float),
        np.array([7.6, 4.3, 3.5, 0, 1.4], dtype=float),
        np.array([8.8, 4.3, 4.7, 1.4, 0], dtype=float),
    ], 
    columns=['A', 'B', 'C', 'D', 'E'], 
    index=idx)
df

,A,B,C,D,E
A,0.0,5.8,5.8,7.6,8.8
B,5.8,0.0,2.4,4.3,4.3
C,5.8,2.4,0.0,3.5,4.7
D,7.6,4.3,3.5,0.0,1.4
E,8.8,4.3,4.7,1.4,0.0


---

# Qubo Problem creation

Using the Dwave's Ocean SDK, we'll map these points to a QUBO problem. The idea, is to create a model wich each variable is correlated with a timestamp and a poistion.

$$ \sum^{4}_{t=1} \sum^{4}_{\substack{i=0\\ \text{if t $\neq$ 1}\\ \text{else i=1}}} \sum^{4}_{\substack{j=0 \\ \text{if t $\neq$ 1}\\ \text{else j=1} \\ \& \\ j \neq  i}} Q_{ij} p_{tij} + P(\sum_{t,i,j}p_{tij} - 4) + P(\sum^{4}_{i=0}\sum^{4}_{\substack{j=0 \\ j \neq  i}}(\sum^{4}_{t=1}p_{tij} - 1))$$

Each $p_{ti}$ is a binary variable.

In [128]:
steps = df.shape[0]
P = 10

linear_terms = {}

every_timestep_sum_up_to_one = [[] for _ in range(steps-1)]
only_one_movement_per_step = [[] for _ in range(steps)]


for t in range(1, steps):
    if(t == 1):
        row = df.iloc[0].tolist()    
        for next_point in range(1, steps):
            val = row[next_point]
            term = f'p_10{next_point}'
            linear_terms[term] = val
            every_timestep_sum_up_to_one[t-1].append((term, 1))
            only_one_movement_per_step[next_point].append((term, 1))
            
    else:
        #for prev in range((1 if t==2 else 0), steps):
        for prev in range(steps):
            row = df.iloc[prev].tolist()
            for next_point in range(steps):
                if(prev == next_point):
                    continue
                val = row[next_point]
                term = f'p_{t}{prev}{next_point}'
                linear_terms[term] = val   
                every_timestep_sum_up_to_one[t-1].append((term, 1))
                only_one_movement_per_step[next_point].append((term, 1))
                

#linear_terms, every_timestep_sum_up_to_one, only_one_movement_per_step

In [167]:
steps = df.shape[0]

labels = idx.tolist()

quadratic_terms = {}

every_timestep_sum_up_to_two = []
every_part_sum_up_to_one = []

for t in range(1, steps):

    every_timestep_sum_up_to_two.append([])
    every_part_sum_up_to_one.append([])
    every_part_sum_up_to_one.append([])

    for i in labels:
        for j in labels:
            if(i == j):
                continue
            term1 = f'{i}1_{t}'
            term2 = f'{j}2_{t}'

            every_timestep_sum_up_to_two[-1].append((term1, 1))
            every_timestep_sum_up_to_two[-1].append((term2, 1))
            every_part_sum_up_to_one[-2].append((term1,1))
            every_part_sum_up_to_one[-1].append((term2,1))
            
            quadratic_terms[(term1, term2)] = df[i][j]


In [178]:
P = 30

qubo = BQM(vartype="BINARY")
#qubo.add_linear_from(linear_terms)
qubo.add_quadratic_from(quadratic_terms)

#for constraint in every_timestep_sum_up_to_two:
#    qubo.add_linear_equality_constraint(constraint, P, -2)

for constraint in every_part_sum_up_to_one:
    qubo.add_linear_equality_constraint(constraint, P, -1)

In [179]:
sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample(qubo, num_reads=1000)

In [180]:
result = sampleset.to_pandas_dataframe()
print('Saving CSV file...')
result.to_csv("final-result.csv")

Saving CSV file...


In [181]:
min_energy = result['energy'].min()
print('min energy SimulatedAnnealingSampler(): ', min_energy)

min energy SimulatedAnnealingSampler():  239.9999999998945


In [182]:
lowest_energies = sampleset.lowest().to_pandas_dataframe()
lowest_energies.head()

,A1_1,A1_2,A1_3,A1_4,A2_1,A2_2,A2_3,A2_4,B1_1,B1_2,B1_3,B1_4,B2_1,B2_2,B2_3,B2_4,C1_1,C1_2,C1_3,C1_4,C2_1,C2_2,C2_3,C2_4,D1_1,D1_2,D1_3,D1_4,D2_1,D2_2,D2_3,D2_4,E1_1,E1_2,E1_3,E1_4,E2_1,E2_2,E2_3,E2_4,energy,num_occurrences
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,240.0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,240.0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,240.0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,240.0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,240.0,1


In [127]:
valid_paths = []
labels = lowest_energies.columns[:-2]  

for i in range(lowest_energies.shape[0]):
    data = lowest_energies.iloc[i][labels].to_frame().T
    selected_points = data.columns[(data==1).any()].tolist()

    follow_the_path = True
    for j in range(1,steps):
        point = selected_points[j]
        last_point = selected_points[j-1]
        if(point[3] != last_point[4]):
            follow_the_path = False
            break

    if(follow_the_path):
        valid_paths.append(selected_points)
    
valid_paths

[]